In [1]:
import re
import pandas as pd
import numpy as np
import sqlite3 

from sqlalchemy import create_engine
from pathlib import Path
from datetime import datetime

In [2]:
data_folder = Path('..', 'data')

data_files = list(data_folder.glob("*"))

data_files

[WindowsPath('../data/AFD00009000240006234 copy.txt'),
 WindowsPath('../data/AFD00009000240006234.txt')]

In [3]:
pattern = re.compile("(\d{9})(\d{1})(\d{2})(\d{2})(\d{4})(\d{2})(\d{2})(\w{0,1})(\d{12})([\D]+)")

In [4]:
filename = data_files[0]
print(filename)

lines = None
count_lines = 0
count_register = 0

data = list()

with open(filename, 'r', encoding='ISO8859-1') as file:
    for line in file:
        count_lines += 1
        match = pattern.fullmatch(line)
        if match:
            count_register += 1
            data.append(match.groups())
        else:
            print(line)
        
    
print(count_lines, count_register)

columns = [
    'sequential_number',
    'cod_op',
    'day',
    'month',
    'year',
    'hour',
    'minutes',
    'cod_input',
    'pis',
    'name'
]

frame = pd.DataFrame(data, columns=columns)

del data

# frame['name'] = frame['name'].str.replace('\ {0,}\n{1,}', '')
frame['name'] = frame.name.str.strip('[ \n]')


frame[['name', 'cod_input']] = frame[['name', 'cod_input']].replace('', np.nan)

numeric_attribs = ['sequential_number', 'cod_op', 'day', 'month', 'year', 'hour', 'minutes' ]

frame[numeric_attribs] = frame[numeric_attribs].astype(int)

date_attribs = [ 'year', 'month', 'day', 'hour', 'minutes']

frame['datetime']  = pd.to_datetime(frame[date_attribs])

..\data\AFD00009000240006234 copy.txt
0000000001177778744000166000000000000Cï¿½MARA MUNICIPAL DE SANTO ANTONIO DA PLATINA                                                                                                          000090002400062341809201417022020170220201149

0000000012180920141115177778744000166000000000000Cï¿½MARA MUNICIPAL DE SANTO ANTONIO DA PLATINA                                                                                                          CAMARA MUNICIPAL DE ST. ANT. DA PLATINA                                                             

0000210372200320171315177778744000166000000000000SANTO ANTONIO DA PLATINA CAMARA DE VEREADORES                                                                                                         Cï¿½MARA MUNICIPAL DE SANTO ANTï¿½NIO DA PLATINA - AVENIDA CORONEL OLIVEIRA MOTTA, 715 - CENTRO         

0000210522210320170816177778744000166000000000000Cï¿½MARA MUNICIPAL DE SANTO ANTONIO DA PLATINA                       

In [5]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43156 entries, 0 to 43155
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   sequential_number  43156 non-null  int32         
 1   cod_op             43156 non-null  int32         
 2   day                43156 non-null  int32         
 3   month              43156 non-null  int32         
 4   year               43156 non-null  int32         
 5   hour               43156 non-null  int32         
 6   minutes            43156 non-null  int32         
 7   cod_input          79 non-null     object        
 8   pis                43156 non-null  object        
 9   name               79 non-null     object        
 10  datetime           43156 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int32(7), object(3)
memory usage: 2.5+ MB


In [6]:
frame.head()

,sequential_number,cod_op,day,month,year,hour,minutes,cod_input,pis,name,datetime
0,2,5,18,9,2014,11,15,I,010807995603,MARIA JOSï¿½ DE LIMA CAMPOS MELO,2014-09-18 11:15:00
1,3,5,18,9,2014,11,16,A,010807995603,MARIA JOSï¿½ DE LIMA CAMPOS MELO,2014-09-18 11:16:00
2,4,3,18,9,2014,11,17,NaN,010807995603,NaN,2014-09-18 11:17:00
3,5,3,18,9,2014,11,21,NaN,010807995603,NaN,2014-09-18 11:21:00
4,6,3,18,9,2014,11,22,NaN,010807995603,NaN,2014-09-18 11:22:00


## Identificar todos os servidores e respectivos código PIS

In [7]:
workers = (frame[['name', 'pis']]
           .copy()
           .drop_duplicates(subset=['pis'], 
                            keep='first', 
                            inplace=False,
                            ignore_index=True)
           .sort_values(['name', 'pis'])
          )
    
print(workers.shape)

# workers['name'].replace('', np.nan, inplace=True)

workers

(29, 2)


,name,pis
16,ANA CARLA DOS SANTOS PEREIRA,020637703280
10,ANDERSON SILVA ESTEFANUTO,017058014882
7,DANIELE DE LIMA ALVES,016002684248
22,DIEGO ABDALLA DE OLIVEIRA,013065811536
4,GILIARD ALMEIDA DE GODOI,013979783277
1,LENIR MARIA DE SOUZA CERQUEIRA,017048753097
3,LUCAS PEREIRA VILAS BOAS,020071346877
27,MANOEL SANCHES GARCIA NETO,021065315459
2,MARCO ANTï¿½NIO MARTINS,019004076622
9,MARIA DE LOURDES ENAMI TAKANO,010829150282


In [8]:
engine = create_engine("sqlite:///registros.db", echo=False)

with engine.begin() as connection:
    workers.to_sql('funcionarios', 
                   con=connection, 
                   if_exists='replace',
                   index=False)
    
    frame.to_sql('registros', 
                 con=connection, 
                 if_exists='replace',
                 index=False)

In [9]:
with engine.begin() as connection:
    connection.execute("SELECT DISTINCT name FROM funcionarios").fetchall()

In [10]:
engine.execute("SELECT DISTINCT name FROM funcionarios WHERE name IS NOT NULL").fetchall()

[('ANA CARLA DOS SANTOS PEREIRA',),
 ('ANDERSON SILVA ESTEFANUTO',),
 ('DANIELE DE LIMA ALVES',),
 ('DIEGO ABDALLA DE OLIVEIRA',),
 ('GILIARD ALMEIDA DE GODOI',),
 ('LENIR MARIA DE SOUZA CERQUEIRA',),
 ('LUCAS PEREIRA VILAS BOAS',),
 ('MANOEL SANCHES GARCIA NETO',),
 ('MARCO ANTï¿½NIO MARTINS',),
 ('MARIA DE LOURDES ENAMI TAKANO',),
 ('MARIA JOSï¿½ DE LIMA CAMPOS MELO',),
 ('RAFAEL CRISTIANO DE TOLEDO',),
 ('RENATO LOPES PIRES',),
 ('SILVIA MARIA RAMOS',)]

In [11]:
engine.execute("SELECT max(sequential_number) FROM registros").fetchall()

[(43159,)]

In [12]:
engine.execute("SELECT DISTINCT pis FROM registros WHERE year = 2019 AND month=9").fetchall()

[('013118171501',),
 ('012303844632',),
 ('013979783277',),
 ('012699519512',),
 ('016002684248',),
 ('019004076622',),
 ('021065315459',),
 ('020071346877',),
 ('017048753097',),
 ('020637703280',)]

In [13]:
df = pd.read_sql("SELECT * FROM registros WHERE year = 2019 AND month=9 AND pis = '013979783277' ", 
                 engine,
                parse_dates='datetime')

# df.set_index('datetime', inplace=True)

In [14]:
df

,sequential_number,cod_op,day,month,year,hour,minutes,cod_input,pis,name,datetime
0,40356,3,2,9,2019,8,0,None,013979783277,None,2019-09-02 08:00:00
1,40365,3,2,9,2019,12,0,None,013979783277,None,2019-09-02 12:00:00
2,40377,3,2,9,2019,13,11,None,013979783277,None,2019-09-02 13:11:00
3,40384,3,2,9,2019,17,2,None,013979783277,None,2019-09-02 17:02:00
4,40393,3,2,9,2019,19,38,None,013979783277,None,2019-09-02 19:38:00
...,...,...,...,...,...,...,...,...,...,...,...
78,41064,3,30,9,2019,12,3,None,013979783277,None,2019-09-30 12:03:00
79,41074,3,30,9,2019,13,12,None,013979783277,None,2019-09-30 13:12:00
80,41078,3,30,9,2019,17,0,None,013979783277,None,2019-09-30 17:00:00
81,41088,3,30,9,2019,19,38,None,013979783277,None,2019-09-30 19:38:00


In [15]:
# df.resample('d').min()

In [16]:
df['rank'] = df.groupby(df.datetime.dt.day)['datetime'].rank().astype(int)

In [17]:
df.head(10)

,sequential_number,cod_op,day,month,year,hour,minutes,cod_input,pis,name,datetime,rank
0,40356,3,2,9,2019,8,0,None,013979783277,None,2019-09-02 08:00:00,1
1,40365,3,2,9,2019,12,0,None,013979783277,None,2019-09-02 12:00:00,2
2,40377,3,2,9,2019,13,11,None,013979783277,None,2019-09-02 13:11:00,3
3,40384,3,2,9,2019,17,2,None,013979783277,None,2019-09-02 17:02:00,4
4,40393,3,2,9,2019,19,38,None,013979783277,None,2019-09-02 19:38:00,5
5,40396,3,2,9,2019,21,14,None,013979783277,None,2019-09-02 21:14:00,6
6,40405,3,3,9,2019,8,2,None,013979783277,None,2019-09-03 08:02:00,1
7,40412,3,3,9,2019,12,0,None,013979783277,None,2019-09-03 12:00:00,2
8,40422,3,3,9,2019,13,5,None,013979783277,None,2019-09-03 13:05:00,3
9,40433,3,3,9,2019,17,2,None,013979783277,None,2019-09-03 17:02:00,4
